This is the GPyTorch version of a custom bayesian optimizer being developed for IDEABio

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import qmc
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, Matern, RationalQuadratic, Kernel, Hyperparameter
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

import GPy
import gpytorch

Below is an old GPy attempt at a category overlap kernel for reference (can delete later)

In [5]:
class CategoryOverlapKernelGPy(GPy.kern.Kern):
    """GPy implementation of a categorical overlap kernel."""

    def __init__(self, input_dim, variance=1.0, active_dims=None, name='catoverlap'):
        super().__init__(input_dim, active_dims=active_dims, name=name)
        self.variance = GPy.core.parameterization.Param('variance', variance)
        self.link_parameter(self.variance)

    def K(self, X, X2=None):
        if X2 is None:
            X2 = X

        diff = X[:, None] - X2[None, :]
        diff[np.where(np.abs(diff))] = 1  # Mark different categories
        k_cat = self.variance * (1 - np.mean(diff, axis=-1))  # Normalize overlap
        return k_cat



Just a test to ensure kernel matrix looks correct for categorical data

In [7]:
X_test = np.array([[4], [2], [2], [1], [0], [8]]) #each row is a category

# Create kernel and compute the kernel matrix
gpy_kernel = CategoryOverlapKernelGPy(input_dim=1, variance=1.0)
K_gpy = gpy_kernel.K(X_test)
print("GPy Kernel Matrix:\n", K_gpy)



GPy Kernel Matrix:
 [[1. 0. 0. 0. 0. 0.]
 [0. 1. 1. 0. 0. 0.]
 [0. 1. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]]


Below is a chatgpt generated model for a simple GP using Matern kernel for me to study how this all works

In [17]:
import torch
import gpytorch

# Define a simple Gaussian Process model using GPyTorch
class SimpleGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super().__init__(train_x, train_y, likelihood)
        
        # Define the mean function (constant mean assumes data has an underlying constant trend)
        self.mean_module = gpytorch.means.ConstantMean()
        
        # Define the covariance function (Matern kernel models spatial correlations with smoothness parameter nu=2.5)
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.MaternKernel(nu=2.5))
    
    def forward(self, x):
        # Compute the mean and covariance matrix for input data
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        
        # Return a Gaussian distribution with computed mean and covariance
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# Wrapper class for training and prediction using the GP model
class SimpleGPR:
    def __init__(self, input_dim, noise_var=1e-5):
        self.input_dim = input_dim  # Number of input dimensions (features)
        self.likelihood = gpytorch.likelihoods.GaussianLikelihood()  # Gaussian likelihood models observation noise
        self.model = None  # Placeholder for GP model
        self.X, self.y = None, None  # Storage for training data
        self.noise_var = noise_var  # Noise variance
    
    def fit(self, X, y):
        # Store training data (ensure y is a 1D tensor)
        self.X, self.y = X, y.squeeze()
        
        # Initialize the Gaussian Process model
        self.model = SimpleGPModel(self.X, self.y, self.likelihood)
        
        # Set the model and likelihood to training mode
        self.model.train()
        self.likelihood.train()
        
        # Use Adam optimizer to maximize marginal likelihood
        optimizer = torch.optim.Adam(self.model.parameters(), lr=0.1)
        
        # Define the loss function (negative marginal log likelihood)
        mll = gpytorch.mlls.ExactMarginalLogLikelihood(self.likelihood, self.model)
        
        # Training loop (optimize parameters for 50 iterations)
        for _ in range(50):
            optimizer.zero_grad()  # Reset gradients
            output = self.model(self.X)  # Compute GP predictions
            loss = -mll(output, self.y)  # Compute negative log likelihood
            loss.backward()  # Compute gradients
            optimizer.step()  # Update model parameters
    
    def predict(self, X):
        # Set the model and likelihood to evaluation mode
        self.model.eval()
        self.likelihood.eval()
        
        # Disable gradient computation for prediction
        with torch.no_grad(), gpytorch.settings.fast_pred_var():
            preds = self.model(X)  # Get GP predictions
        
        # Return mean prediction and standard deviation
        return preds.mean, preds.variance.sqrt()

# Example Usage
if __name__ == "__main__":
    # Example training data (3 continuous input features)
    X = torch.tensor([
        [0.5, 0.2, 0.1],
        [0.2, 0.8, 0.3],
        [0.7, 0.5, 0.6]
    ])  # Training inputs
    y = torch.tensor([0.3, 0.7, 0.5])  # Corresponding target values
    
    # Instantiate and train the Gaussian Process Regression model
    gpr = SimpleGPR(input_dim=3)
    gpr.fit(X, y)
    
    # New test points for prediction (3 features each)
    new_X = torch.tensor([
        [0.6, 0.3, 0.4],
        [0.3, 0.7, 0.2]
    ])
    mean, std = gpr.predict(new_X)
    
    # Print mean and standard deviation of predictions
    print(f"Mean: {mean}, Std: {std}")



Mean: tensor([0.4926, 0.4977]), Std: tensor([0.0669, 0.0671])


This below is the same as above but with chatgpt trying to integrate the cat overlap. it will clearly not work, but could potentially be useful for further understanding gpytorch

In [18]:
import torch
import gpytorch

# Define a custom categorical kernel using category overlap
class CategoricalKernel(gpytorch.kernels.Kernel):
    def __init__(self, **kwargs):
        super().__init__(has_lengthscale=False, **kwargs)  # No lengthscale for categorical kernel
        self.overlap_variance = torch.nn.Parameter(torch.tensor(1.0))  # Trainable variance parameter

    def forward(self, x1, x2, **params):
        # Compute category overlap matrix (1 if same, 0 if different)
        overlap_matrix = (x1[:, None] == x2[None, :]).float().sum(dim=-1)
        return self.overlap_variance * overlap_matrix

# Define a Gaussian Process model using both Matern kernel (for continuous) and categorical kernel
class SimpleGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, cat_dim):
        super().__init__(train_x, train_y, likelihood)
        
        # Define the mean function (assumes an underlying constant trend)
        self.mean_module = gpytorch.means.ConstantMean()
        
        # Define the covariance function
        self.continuous_covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.MaternKernel(nu=2.5))
        self.categorical_covar_module = CategoricalKernel()
        
        # Combine continuous and categorical kernels additively
        self.covar_module = self.continuous_covar_module + self.categorical_covar_module
    
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# Wrapper class for training and prediction using the GP model
class SimpleGPR:
    def __init__(self, input_dim=3, categorical_dim=1, noise_var=1e-5):
        self.input_dim = input_dim  # Number of continuous input dimensions
        self.categorical_dim = categorical_dim  # Number of categorical variables
        self.likelihood = gpytorch.likelihoods.GaussianLikelihood()  # Gaussian likelihood models observation noise
        self.model = None  # Placeholder for GP model
        self.X, self.y = None, None  # Storage for training data
        self.noise_var = noise_var  # Noise variance
    
    def fit(self, X, y):
        self.X, self.y = X, y.squeeze()
        
        # Initialize the Gaussian Process model
        self.model = SimpleGPModel(self.X, self.y, self.likelihood, self.categorical_dim)
        
        # Set the model and likelihood to training mode
        self.model.train()
        self.likelihood.train()
        
        # Use Adam optimizer to maximize marginal likelihood
        optimizer = torch.optim.Adam(self.model.parameters(), lr=0.1)
        
        # Define the loss function (negative marginal log likelihood)
        mll = gpytorch.mlls.ExactMarginalLogLikelihood(self.likelihood, self.model)
        
        # Training loop (optimize parameters for 50 iterations)
        for _ in range(50):
            optimizer.zero_grad()  # Reset gradients
            output = self.model(self.X)  # Compute GP predictions
            loss = -mll(output, self.y)  # Compute negative log likelihood
            loss.backward()  # Compute gradients
            optimizer.step()  # Update model parameters
    
    def predict(self, X):
        self.model.eval()
        self.likelihood.eval()
        
        with torch.no_grad(), gpytorch.settings.fast_pred_var():
            preds = self.model(X)
        
        return preds.mean, preds.variance.sqrt()

# Example Usage
if __name__ == "__main__":
    # Example training data (3 continuous input features + 1 categorical variable)
    X = torch.tensor([
        [0.5, 0.2, 0.1, 0],
        [0.2, 0.8, 0.3, 1],
        [0.7, 0.5, 0.6, 0]
    ])  # Training inputs (last column is categorical)
    y = torch.tensor([0.3, 0.7, 0.5])  # Corresponding target values
    
    # Instantiate and train the Gaussian Process Regression model
    gpr = SimpleGPR(input_dim=3, categorical_dim=1)
    gpr.fit(X, y)
    
    # New test points for prediction (3 continuous features + 1 categorical)
    new_X = torch.tensor([
        [0.6, 0.3, 0.4, 1],
        [0.3, 0.7, 0.2, 0]
    ])
    mean, std = gpr.predict(new_X)
    
    print(f"Mean: {mean}, Std: {std}")


 c:\Users\uqkmuroi\.virtualenvs\EnzymeKineticGPR-62lwjgN4\Lib\site-packages\linear_operator\utils\cholesky.py:40: NumericalWarning:A not p.d., added jitter of 1.0e-06 to the diagonal
 c:\Users\uqkmuroi\.virtualenvs\EnzymeKineticGPR-62lwjgN4\Lib\site-packages\linear_operator\utils\cholesky.py:40: NumericalWarning:A not p.d., added jitter of 1.0e-05 to the diagonal
 c:\Users\uqkmuroi\.virtualenvs\EnzymeKineticGPR-62lwjgN4\Lib\site-packages\linear_operator\utils\cholesky.py:40: NumericalWarning:A not p.d., added jitter of 1.0e-04 to the diagonal


NotPSDError: Matrix not positive definite after repeatedly adding jitter up to 1.0e-04.

Below is an attempt at making the GPyTorch model work with the cocabo concept

In [10]:
import torch
import gpytorch

class CategoryOverlapKernel(gpytorch.kernels.Kernel):
    """Custom kernel for cateogrical varaibles using category overlap similarity."""
    
    def __init__(self, **kwargs):
        super().__init__(has_lengthsacle = False, **kwargs)
    
    def forward(self, x1, x2, diag = False, **params):
        if diag:
            return torch.ones(x1.shape[0], dtype=x1.dtype, device=x1.device)
        
        #check if categorical values are teh same (Kronecker delta function)
        overlap = x1[:, None] == x2[None, :].float()
        return overlap
    
    
class CombinedKernel(gpytorch.kernels.Kernel):
    """Combined Matern kernel and Category Overlap Kernel"""

    def __init__(self, matern_nu=2.5, **kwargs):
        super().__init__(**kwargs)
        self.matern_kernel = gpytorch.kernels.MaternKernel(nu=matern_nu)
        self.category_kernel = CategoryOverlapKernel()

    def forward(self, x1, x2, diag = False, **params):
        #split cont and cat features
        x1_cont, x1_cat = x1[..., :-1], x1[..., -1]
        x2_cont, x2_cat = x2[..., :-1], x2[..., -1]

        # computer kernel values
        matern_val = self.matern_kernel(x1_cont, x2_cont, diag=diag, **params)
        category_val = self.category_kernel(x1_cat, x2_cat, diag=diag, **params)

        #combine kernels
        return matern_val * category_val
    
class CustomGPModel(gpytorch.models.ExactGP):
    def __init__ (self, train_x, train_y, likelihood):
        super().__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = CombinedKernel()

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
    

Below is a boiler plate for a sci-kit learn implementation, technically I should do both for good practice, but ultimately the team will probably want to develop in GPyTorch for consistency, longevity, and for future development.

In [11]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

class SimpleCoCaBO:
    def __init__(self, continuous_dim, categorical_dim, kernel=None, noise_var=1e-5):
        # Initialize the model with dimensions of continuous and categorical variables
        self.continuous_dim = continuous_dim
        self.categorical_dim = categorical_dim
        
        # Define the kernel for the Gaussian Process (RBF + constant kernel)
        self.kernel = kernel if kernel else C(1.0, (1e-4, 1e1)) * RBF(1.0, (1e-4, 1e1))
        
        # Initialize the Gaussian Process Regressor
        self.gpr = GaussianProcessRegressor(kernel=self.kernel, alpha=noise_var)

        # Storage for past data
        self.X = []  # Stores continuous + categorical variables
        self.y = []  # Stores corresponding objective function values

    def fit(self, X_cont, X_cat, y):
        """Fit the Gaussian Process model on both continuous and categorical data."""
        # Combine continuous and categorical data
        X_combined = np.hstack((X_cont, X_cat))
        
        # Fit the Gaussian Process Regressor model
        self.gpr.fit(X_combined, y)
        
        # Store the data for future optimization
        self.X.extend(X_combined)
        self.y.extend(y)

    def predict(self, X_cont, X_cat):
        """Predict mean and variance for new points."""
        X_combined = np.hstack((X_cont, X_cat))
        mean, std = self.gpr.predict(X_combined, return_std=True)
        return mean, std

    def ucb(self, X_cont, X_cat, kappa=2.0):
        """Upper Confidence Bound (UCB) acquisition function."""
        mean, std = self.predict(X_cont, X_cat)
        ucb_values = mean + kappa * std
        return ucb_values

    def optimize(self, X_cont, X_cat, kappa=2.0):
        """Optimize the acquisition function (UCB)."""
        ucb_values = self.ucb(X_cont, X_cat, kappa)
        best_idx = np.argmax(ucb_values)  # Select the index with the highest UCB value
        return X_cont[best_idx], X_cat[best_idx]

# Example usage
if __name__ == "__main__":
    # Example continuous and categorical variables
    X_cont = np.array([[0.5], [0.2], [0.7]])  # Example continuous variables
    X_cat = np.array([[0], [1], [0]])  # Example categorical variables (just encoded as 0 or 1)
    y = np.array([0.3, 0.7, 0.5])  # Objective values

    # Instantiate the SimpleCoCaBO object
    optimizer = SimpleCoCaBO(continuous_dim=1, categorical_dim=1)

    # Fit the model to the data
    optimizer.fit(X_cont, X_cat, y)

    # Predict UCB values for new points
    new_cont = np.array([[0.6], [0.3]])  # New continuous points to evaluate
    new_cat = np.array([[1], [0]])  # New categorical points

    best_cont, best_cat = optimizer.optimize(new_cont, new_cat)
    print(f"Best continuous: {best_cont}, Best categorical: {best_cat}")


Best continuous: [0.6], Best categorical: [1]
